In [ ]:
!pip install ibmcloudsql

In [ ]:
import ibmcloudsql
from pixiedust.display import *

In [ ]:
my_bluemix_apikey = '<your api key here>'
my_instance_crn='<your ibm cloud sql query instance CRN here>'
my_target_cos_url='<your ibm cloud object storage URL (cos://<endpoint>/<bucket>/[prefix]) here>'

In [ ]:
sqlClient = ibmcloudsql.SQLQuery(my_bluemix_apikey, my_instance_crn, my_target_cos_url, client_info='My little SQL notebook')

In [ ]:
result_df = sqlClient.run_sql(
"WITH orders_shipped AS \
  (SELECT OrderID, EmployeeID, (CASE WHEN shippedDate < requiredDate \
                                   THEN 'On Time' \
                                   ELSE 'Late' \
                                END) AS Shipped \
   FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET) \
SELECT e.FirstName, e.LastName, COUNT(o.OrderID) As NumOrders, Shipped \
FROM orders_shipped o, \
     cos://us-geo/sql/employees.parquet STORED AS PARQUET e \
WHERE e.EmployeeID = o.EmployeeID \
GROUP BY e.FirstName, e.LastName, Shipped \
ORDER BY e.LastName, e.FirstName, NumOrders DESC")

In [ ]:
result_df.head(200)

In [ ]:
display(result_df)

In [ ]:
sqlClient.sql_ui_link()

In [ ]:
jobId = sqlClient.submit_sql('SELECT * from cos://us-geo/sql/products.parquet STORED AS PARQUET LIMIT 5')
sqlClient.wait_for_job(jobId)
sqlClient.get_result(jobId).head()

In [ ]:
sqlClient.get_jobs().head(200)